In [1]:
# !pip install ffmpy
# !pip install m3u8
# !pip install earth_cam_downloader 

In [2]:
"""
M3U8 Downloader
Download the ts files according to the given m3u8 file.
"""
import argparse
import os

import ffmpy
import m3u8


class M3U8Downloader:
    """M3U8 Downloader Class"""

    def __init__(
        self,
        uri,
        timeout=None,
        headers=None,
        select_first_playlist=False,
        ffmpeg_path="ffmpeg",
        ffmpeg_loglevel="quiet",
    ):
        """Initialize a M3U8 Downloader.
        Args:
            uri (:obj:`str`): The URI of the m3u8 file.
            timeout (:obj:`int`, optional): The timeout used when loading
                from uri. Defaults to ``None``.
            headers (:obj:`list` of :obj:`str`, optional): The headers used
                when loading from uri. Defaults to ``None``.
            ffmpeg_path (:obj:`str`, optional): The path to ffmpeg executable.
                Defaults to ``ffmpeg``.
            ffmpeg_loglevel (:obj:`str`, optional): The logging level of
                ffmpeg. Defaults to ``quiet``.
            select_first_playlist(:obj:`boolean`, optional): Whether or not to simply select
                the first playlist for a variant m3u8 file.
        """
        if not headers:
            headers = {}

        self.uri = uri
        self.ffmpeg_path = ffmpeg_path
        self.ffmpeg_loglevel = ffmpeg_loglevel
        self.select_first_playlist = select_first_playlist
        self.m3u8 = m3u8.load(uri=uri, timeout=timeout, headers=headers)

    def download(self, output="output.ts"):
        """Start downloading and merging with the given m3u8 file.
        Args:
            output (:obj:`str`): The path to output. Defaults to ``output.ts``
        """
        if self.m3u8.is_variant:
            print("INFO: There are multiple m3u8 files listed in this file.")
            try:
                if self.select_first_playlist:
                    print("INFO: Simply selecting the first playlist to download...")
                    fetch_index = 0
                else:
                    print()
                    for index, playlist in enumerate(self.m3u8.playlists):
                        self._print_stream_info(playlist, index)
                    fetch_index = int(input("Index> "))

                downloader = M3U8Downloader(
                    self.m3u8.playlists[fetch_index].absolute_uri,
                    ffmpeg_path=self.ffmpeg_path,
                    ffmpeg_loglevel=self.ffmpeg_loglevel,
                )
                downloader.download(output)
            except (ValueError, IndexError):
                print("ERROR: Invalid index.")

        else:
            dirname = os.path.dirname(output)
            if dirname:
                os.makedirs(os.path.dirname(output), exist_ok=True)

            ffmpeg_cmd = ffmpy.FFmpeg(
                self.ffmpeg_path,
                "-y -loglevel {}".format(self.ffmpeg_loglevel),
                inputs={self.uri: None},
                outputs={output: "-c copy"},
            )
            print("INFO: Start downloading and merging with ffmpeg...")
            print(ffmpeg_cmd.cmd)

            ffmpeg_cmd.run()

    @staticmethod
    def _print_stream_info(playlist, index=0):
        print("INDEX: " + str(index))

        stream_info = playlist.stream_info
        if stream_info.bandwidth:
            print("Bandwidth: {}".format(stream_info.bandwidth))
        if stream_info.average_bandwidth:
            print("Average bandwidth: {}".format(stream_info.average_bandwidth))
        if stream_info.program_id:
            print("Program ID: {}".format(stream_info.program_id))
        if stream_info.resolution:
            print("Resolution: {}".format(stream_info.resolution))
        if stream_info.codecs:
            print("Codecs: {}".format(stream_info.codecs))
        print()


In [5]:
# !/usr/bin/env python3

import time
from datetime import datetime

from earth_cam_downloader import EarthCamDownloader

# time to start downloading archived footage from
START = datetime(2020, 7, 9, 0, 0, 0)

# time to end downloading archived footage from
END = datetime(2020, 7, 9, 2, 0, 0)

# 6975/2020/07/09

# camera names and ids to download
CAMERAS = {
    "seaside": "6975",
#     "times-square-4k": "hdtimes10",
#     "crossroads": "15559",
#     "south-view": "4017timessquareHD1",
}

FILE_FORMAT = "mp4"

OUTPUT_DIR = "output"


def main():
    started_at = time.time()
    for cam_name, cam_id in CAMERAS.items():
        EarthCamDownloader(
            cam_id, cam_name, START, END, FILE_FORMAT, OUTPUT_DIR
        ).download()
    ended_at = time.time()
    print(f"Downloaded times square files in {round((ended_at -  started_at) / 60, 2)} minutes.")


if __name__ == "__main__":
    main()


INFO: Downloading https://video2archives.earthcam.com/archives/_definst_/MP4:network/6975/2020/07/09/0000.mp4/playlist.m3u8 to output/seaside-2020-07-09-0000.mp4
INFO: There are multiple m3u8 files listed in this file.
INFO: Simply selecting the first playlist to download...
INFO: Start downloading and merging with ffmpeg...
ffmpeg -y -loglevel quiet -i https://video2archives.earthcam.com/archives/_definst_/MP4:network/6975/2020/07/09/0000.mp4/chunklist_w1688242086.m3u8 -c copy output/seaside-2020-07-09-0000.mp4


FFExecutableNotFoundError: Executable 'ffmpeg' not found